In [2]:
## Import all the required libraries

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch import nn
import torch.nn as nn
from torch.utils.data import random_split,DataLoader,Subset
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor,Normalize
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

In [3]:
## RNN
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.Wxh = np.random.randn(hidden_size, input_size)
        self.Whh = np.random.randn(hidden_size, hidden_size)
        self.Why = np.random.randn(output_size, hidden_size)
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        h = np.zeros((self.hidden_size, 1))
        self.inputs = inputs
        self.hs = {}
        for t, x in enumerate(inputs):
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            self.hs[t] = h
        output = np.dot(self.Why, h) + self.by
        return output

In [4]:
## GRU

class GRU:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.Wz = np.random.randn(hidden_size, input_size + hidden_size)
        self.Wr = np.random.randn(hidden_size, input_size + hidden_size)
        self.Wh = np.random.randn(hidden_size, input_size + hidden_size)
        self.Why = np.random.randn(output_size, hidden_size)
        self.bz = np.zeros((hidden_size, 1))
        self.br = np.zeros((hidden_size, 1))
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        h = np.zeros((self.hidden_size, 1))
        self.inputs = inputs
        self.hs = {}
        for t, x in enumerate(inputs):
            z = np.row_stack((h, x))
            z_tilde = sigmoid(np.dot(self.Wz, z) + self.bz)
            r = sigmoid(np.dot(self.Wr, z) + self.br)
            h_bar = np.tanh(np.dot(self.Wh, np.row_stack((r * h, x))) + self.bh)
            h = (1 - z_tilde) * h + z_tilde * h_bar
            self.hs[t] = h
        output = np.dot(self.Why, h) + self.by
        return output

In [5]:
# LSTM
class LSTM:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.Wf = np.random.randn(hidden_size, input_size + hidden_size)
        self.Wi = np.random.randn(hidden_size, input_size + hidden_size)
        self.Wo = np.random.randn(hidden_size, input_size + hidden_size)
        self.Wc = np.random.randn(hidden_size, input_size + hidden_size)
        self.Why = np.random.randn(output_size, hidden_size)
        self.bf = np.zeros((hidden_size, 1))
        self.bi = np.zeros((hidden_size, 1))
        self.bo = np.zeros((hidden_size, 1))
        self.bc = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        h = np.zeros((self.hidden_size, 1))
        c = np.zeros((self.hidden_size, 1))
        self.inputs = inputs
        self.hs = {}
        self.cs = {}
        for t, x in enumerate(inputs):
            z = np.row_stack((h, x))
            f = sigmoid(np.dot(self.Wf, z) + self.bf)
            i = sigmoid(np.dot(self.Wi, z) + self.bi)
            o = sigmoid(np.dot(self.Wo, z) + self.bo)
            c_bar = np.tanh(np.dot(self.Wc, z) + self.bc)
            c = f * c + i * c_bar
            h = o * np.tanh(c)
            self.hs[t] = h
            self.cs[t] = c
        output = np.dot(self.Why, h) + self.by
        return output